In [3]:
import torch
from torch import nn
import torch.nn.functional as F
print(torch.cuda.is_available())

True


In [4]:
from transformers import PreTrainedModel, PretrainedConfig, AutoConfig
class ImageMultiRegressionConfig(PretrainedConfig):
    def __init__(
        self,
        output_size=3,
        init_checkpoint=None,
        **kwargs,
    ):
        self.output_size=output_size
        self.init_checkpoint=init_checkpoint
        super().__init__(**kwargs)


class ImageMultiRegressionModel(PreTrainedModel):
    config_class=ImageMultiRegressionConfig
    def __init__(self, config, loss=nn.MSELoss()):
        super().__init__(config)
        
        self.inner_model = AutoModel.from_pretrained(config.init_checkpoint)
        self.classifier = nn.Linear(self.inner_model.config.hidden_size, config.output_size)
        self.loss=loss
    
    def forward(self, pixel_values, labels=None):
        outputs = self.inner_model(pixel_values=pixel_values)
        cls_output = outputs.last_hidden_state[:, 0, :]  # image embedding
        values = self.classifier(cls_output)
        loss = None
        if labels is not None:
            loss = self.loss(values.view(-1), labels.view(-1))
        return (loss, values) if loss is not None else values

In [6]:
model = ImageMultiRegressionModel.from_pretrained("models/swinv2-base")

In [8]:
model.push_to_hub("mirluvams/swinv2-base-patch4-window16-256-popocatepetl")

[2024-09-18 02:56:52,560] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.4
 [WARNING]  using untested triton version (3.0.0), only 1.0.0 is known to be compatible


/home/miriam/.local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/miriam/.local/lib/python3.10/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


model.safetensors:   0%|          | 0.00/348M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mirluvams/swinv2-base-patch4-window16-256-popocatepetl/commit/4ee912dcd1d6848c911973ec014172699b7fe443', commit_message='Upload model', commit_description='', oid='4ee912dcd1d6848c911973ec014172699b7fe443', pr_url=None, pr_revision=None, pr_num=None)